<a href="https://colab.research.google.com/github/zakkaizzatur/Data-Related-Projects/blob/main/Indonesian_Movie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
import os
os.environ['KAGGLE_USERNAME'] = 'zakkaizzatur'
os.environ['KAGGLE_KEY'] = 'ebfe0360db68f192c2b32c725f0c6b43'

In [3]:
!kaggle datasets download -d antoniuscs/imdb-synopsis-indonesian-movies

  0% 0.00/299k [00:00<?, ?B/s]
100% 299k/299k [00:00<00:00, 44.5MB/s]


In [4]:
!unzip -q imdb-synopsis-indonesian-movies.zip -d .

In [5]:
link = '/content/imdb_indonesian_movies_2.csv'

In [6]:
df = pd.read_csv(link)

In [7]:
df.head()

,judul_film,ringkasan_sinopsis,genre
0,Sunan Kalijaga,Raden Mas Said putra sulung Tumenggung Wilarik...,Drama
1,Gie,Soe Hok Gie adalah seorang aktivis yang hidup ...,Drama
2,Guru Bangsa Tjokroaminoto,Guru Bangsa Tjokroaminoto menceritakan tentang...,Drama
3,POL Movie,POL menceritakan kisah hidup yang luar biasa d...,Drama
4,Sang pencerah,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,Drama


In [8]:
df.drop(columns=['judul_film'], inplace=True)

In [9]:
df.head()

,ringkasan_sinopsis,genre
0,Raden Mas Said putra sulung Tumenggung Wilarik...,Drama
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,Drama
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,Drama
3,POL menceritakan kisah hidup yang luar biasa d...,Drama
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,Drama


In [10]:
df_baru = pd.concat([df, pd.get_dummies(df.genre)], axis=1)

In [11]:
df_baru.drop(columns='genre', inplace=True)

In [12]:
df_baru.head()

,ringkasan_sinopsis,Drama,Horor,Komedi,Laga,Romantis
0,Raden Mas Said putra sulung Tumenggung Wilarik...,1,0,0,0,0
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,1,0,0,0,0
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,1,0,0,0,0
3,POL menceritakan kisah hidup yang luar biasa d...,1,0,0,0,0
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,1,0,0,0,0


In [13]:
sinopsis = df_baru['ringkasan_sinopsis'].values
label = df_baru[['Drama', 'Horor', 'Komedi', 'Laga', 'Romantis']].values

In [14]:
from sklearn.model_selection import train_test_split
sinopsis_latih, sinopsis_test, label_latih, label_test = train_test_split(sinopsis, label, test_size=0.2)

In [15]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
 
tokenizer = Tokenizer(num_words=6000, oov_token='x')
tokenizer.fit_on_texts(sinopsis_latih) 
tokenizer.fit_on_texts(sinopsis_test)
 
sekuens_latih = tokenizer.texts_to_sequences(sinopsis_latih)
sekuens_test = tokenizer.texts_to_sequences(sinopsis_test)
 
padded_latih = pad_sequences(sekuens_latih) 
padded_test = pad_sequences(sekuens_test)

In [19]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(tokenizer.index_word), output_dim=16),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(5, activation='softmax')
])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [20]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 16)          218832    
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                20736     
_________________________________________________________________
dense_3 (Dense)              (None, 128)               8320      
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 5)                

In [23]:
num_epochs = 30
history = model.fit(padded_latih, label_latih, epochs=num_epochs, 
                    validation_data=(padded_test, label_test), verbose=2)

Epoch 1/30
26/26 - 5s - loss: 0.0023 - accuracy: 1.0000 - val_loss: 8.7173 - val_accuracy: 0.3284
Epoch 2/30
26/26 - 5s - loss: 0.0649 - accuracy: 0.9851 - val_loss: 6.5681 - val_accuracy: 0.3284
Epoch 3/30
26/26 - 5s - loss: 0.1122 - accuracy: 0.9652 - val_loss: 5.6221 - val_accuracy: 0.2886
Epoch 4/30
26/26 - 5s - loss: 0.0739 - accuracy: 0.9826 - val_loss: 5.2514 - val_accuracy: 0.3234
Epoch 5/30
26/26 - 5s - loss: 0.0411 - accuracy: 0.9913 - val_loss: 5.1157 - val_accuracy: 0.3731
Epoch 6/30
26/26 - 5s - loss: 0.0180 - accuracy: 0.9950 - val_loss: 5.4779 - val_accuracy: 0.3383
Epoch 7/30
26/26 - 5s - loss: 0.0055 - accuracy: 0.9988 - val_loss: 5.6172 - val_accuracy: 0.3433
Epoch 8/30
26/26 - 5s - loss: 0.0042 - accuracy: 1.0000 - val_loss: 5.8004 - val_accuracy: 0.3383
Epoch 9/30
26/26 - 5s - loss: 0.0156 - accuracy: 0.9975 - val_loss: 6.1979 - val_accuracy: 0.3234
Epoch 10/30
26/26 - 5s - loss: 0.0094 - accuracy: 0.9963 - val_loss: 6.1740 - val_accuracy: 0.3383
Epoch 11/30
26/26 -